In [ ]:
import math
from numpy import linalg as LA

@njit()
def fit_J(w, res, expn, etal, sigma):
    for i in range(len(etal)):
        res += etal[i] / (expn[i] + sigma * 1.j * w)


@njit()
def fit_t(t, res, expn, etal):
    for i in range(len(etal)):
        res += etal[i] * np.exp(-expn[i] * t)
    return res

In [ ]:
nmod = 4
lams1 = np.e
lams2 = np.e
gams1 = 1
gams2 = 1
omgs1 = 0
omgs2 = 0
temp = 0.0000000001
beta = int(1 / temp)
mu_x = 0

def gen_jw(w):
    return w * lams1 * np.exp(- gams1 * np.abs(w))

#     jw = np.zeros_like(w)
#     index = (w<=gams1) & (w>=-gams1)
#     jw[index] = lams1 * np.sqrt(gams1**2 - w[index]**2)
#     return jw

len_ = 10000
spe_wid = 10
w = np.linspace(-spe_wid, spe_wid, len_)

sigma = 1
jw = gen_jw(w)
jw1 = jw  / (1 - np.exp(- beta * w))
plt.plot(w, jw)
plt.plot(w, jw1)
plt.xlim(-10, 10)

In [ ]:
jw

In [ ]:
n = 2000
scale = 80

scale_fft = 1000
n_fft = 10000000

n_rate = (scale_fft * scale/ (4 * n))
print(n_rate)
n_rate = int(n_rate)

w = np.linspace(0, scale_fft * np.pi, n_fft + 1)[:-1]
dw = w[1] - w[0]

jw = gen_jw(w)
cw1 = jw / (1 - np.exp(-beta * (w - mu_x)))
cw2 = jw / (1 - np.exp(beta * (w - mu_x)))
del jw

cw1[0] = (cw1[1] + cw2[1]) / 4
cw2[0] = (cw1[1] + cw2[1]) / 4
fft_ct = (np.fft.fft(cw1) * dw + np.fft.ifft(cw2) * len(cw2) * dw) / np.pi
fft_t = 2 * np.pi * np.fft.fftfreq(len(cw1), dw)
del cw1, cw2

fft_ct = fft_ct[(scale>=fft_t) & (fft_t >= 0)][::n_rate]
fft_t = fft_t[(scale>=fft_t) & (fft_t >= 0)][::n_rate]

t = fft_t
res_t = np.zeros(len(t), dtype=complex)

In [ ]:
len_ = 100000
spe_wid = 100
w = np.linspace(-spe_wid, spe_wid, len_)

sigma = 1
jw = gen_jw(w)
jw1 = jw / (1 - np.exp(-beta * w))
# plt.plot(w, jw)
plt.plot(w, jw1)
# fft_ct[0] = fft_ct[0] / 2
# plt.plot(2 * np.pi * np.fft.fftfreq(len(fft_ct), fft_t[1] - fft_t[0]),
#          len(fft_ct) * (fft_t[1] - fft_t[0]) * np.fft.ifft(fft_ct))
plt.xlim(-5, 5)
# fft_ct[0] = fft_ct[0] * 2

fft_ct = np.e * 2 / np.pi * (1 - fft_t * fft_t) / 2 / (1 + fft_t * fft_t) / (
    1 + fft_t * fft_t) - 1.j* 2 / np.pi * np.e * fft_t / (1 + fft_t * fft_t) / (
        1 + fft_t * fft_t)
plt.plot(2 * np.pi * np.fft.fftfreq(len(fft_ct), fft_t[1] - fft_t[0]),
         len(fft_ct) * (fft_t[1] - fft_t[0]) * np.fft.ifft(fft_ct), 'r')
fft_ct

In [ ]:
plt.plot(fft_t, np.imag(fft_ct))
plt.plot(fft_t, np.real(fft_ct))
plt.xlim(0, 10)
print(fft_ct[:100])
print(fft_ct[-100:])

In [ ]:
n_sample = n + 1
n_gamma_l = [8]
h = np.imag(fft_ct)
H = np.zeros((n_sample, n_sample))
for i in range(n_sample):
    H[i, :] = h[i:n_sample + i]
sing_vs, Q = LA.eigh(H)
# del H
phase_mat = np.diag(
    [np.exp(-1j * np.angle(sing_v) / 2.0) for sing_v in sing_vs])
vs = np.array([np.abs(sing_v) for sing_v in sing_vs])
Qp = np.dot(Q, phase_mat)
sort_array = np.argsort(vs)[::-1]
vs = vs[sort_array]
Qp = (Qp[:, sort_array])
vs = vs[:20]
Qp = Qp[:, :20]

for n_gamma in n_gamma_l:
    print("len of gamma", n_gamma)
    gamma = np.roots(Qp[:, n_gamma][::-1])
    gamma_new = gamma[np.argsort(np.abs(gamma))[:n_gamma]]
    t_imag = 2 * n * np.log(gamma_new)
    gamma_m = np.zeros((n_sample * 2 - 1, n_gamma), dtype=complex)
    for i in range(n_gamma):
        for j in range(n_sample * 2 - 1):
            gamma_m[j, i] = gamma_new[i]**j
    omega_imag = np.dot(LA.inv(np.dot(np.transpose(gamma_m), gamma_m)),
                        np.dot(np.transpose(gamma_m), np.transpose(h)))

    res_t = np.zeros(len(t), dtype=complex)
    fit_t(fft_t, res_t, -t_imag / scale, omega_imag)
    plt.plot(fft_t, np.imag(fft_ct) - res_t)
    plt.savefig("imag_{}.pdf".format(n_gamma))
    plt.clf()

h = np.real(fft_ct)
H = np.zeros((n_sample, n_sample))
for i in range(n_sample):
    H[i, :] = h[i:n_sample + i]
sing_vs, Q = LA.eigh(H)
# del H
phase_mat = np.diag(
    [np.exp(-1j * np.angle(sing_v) / 2.0) for sing_v in sing_vs])
vs = np.array([np.abs(sing_v) for sing_v in sing_vs])
Qp = np.dot(Q, phase_mat)
sort_array = np.argsort(vs)[::-1]
vs = vs[sort_array]
Qp = (Qp[:, sort_array])
vs = vs[:20]
Qp = Qp[:, :20]

for n_gamma in n_gamma_l:
    print("len of gamma", n_gamma)
    gamma = np.roots(Qp[:, n_gamma][::-1])
    gamma_new = gamma[np.argsort(np.abs(gamma))[:n_gamma]]
    t_real = 2 * n * np.log(gamma_new)
    gamma_m = np.zeros((n_sample * 2 - 1, n_gamma), dtype=complex)
    for i in range(n_gamma):
        for j in range(n_sample * 2 - 1):
            gamma_m[j, i] = gamma_new[i]**j
    omega_real = np.dot(LA.inv(np.dot(np.transpose(gamma_m), gamma_m)),
                        np.dot(np.transpose(gamma_m), np.transpose(h)))

    res_t = np.zeros(len(t), dtype=complex)
    fit_t(fft_t, res_t, -t_real / scale, omega_real)
    plt.plot(fft_t, np.real(fft_ct) - res_t)
    plt.savefig("real_{}.pdf".format(n_gamma))
    plt.clf()

In [ ]:
np.log(gamma_new)

In [ ]:
np.angle(gamma_new)

In [ ]:
# np.dot(phase_mat, H) - np.dot(H, phase_mat)

In [ ]:
etal1 = np.append(1.j * omega_imag, omega_real)
etar1 = np.append(np.conjugate(1.j * omega_imag), np.conjugate(omega_real))
etaa1 = np.append(np.abs(omega_imag), np.abs(omega_real))
expn1 = np.append(-t_imag / scale, -t_real / scale)
filter_ = 1e-3
expn1 = expn1[np.abs(etal1) > filter_]
etal1 = etal1[np.abs(etal1) > filter_]
len_ = 100000
spe_wid = 50
w = np.linspace(-spe_wid, spe_wid, len_)
phixx = gen_jw(w) / (1 - np.exp(- beta * w))
res_J1 = np.zeros(len(w), dtype=complex)
fit_J(w, res_J1, expn1, etal1, -1)
plt.plot(w, (phixx.real))
plt.plot(w, (res_J1.real), "--")
plt.xlim(-5, 15)
plt.savefig("res_ohmic.pdf")
plt.show()

len_ = 1000000
spe_wid = 0.2
w = np.linspace(-spe_wid, spe_wid, len_)
phixx = gen_jw(w) / (1 - np.exp(- beta * w))
res_J1 = np.zeros(len(w), dtype=complex)
fit_J(w, res_J1, expn1, etal1, -1)
plt.plot(w, (phixx.real-res_J1.real), "--")